In [1]:
%reload_ext autoreload
%autoreload 2

from comet_ml import Experiment
import h5py
import matplotlib.pyplot as plt
import numpy as np
import argparse
import importlib
import random
import os
from algorithms.server.server import Server
from algorithms.trainmodel.models import *
from utils.plot_utils import *
import torch
torch.manual_seed(0)

def main(experiment, dataset, algorithm, model, batch_size, learning_rate, alpha, eta, L, rho, num_glob_iters,
         local_epochs, optimizer, numedges, times, commet, gpu):

    device = torch.device("cuda:{}".format(gpu) if torch.cuda.is_available() and gpu != -1 else "cpu")

    for i in range(times):
        print("---------------Running time:------------",i)

        # Generate model
        if(model == "mclr"):
            if(dataset == "human_activity"):
                model = Mclr_Logistic(561,6).to(device), model
            else:
                model = Mclr_Logistic().to(device), model

        if(model == "linear_regression"):
            model = Linear_Regression(40,1).to(device), model

        if model == "logistic_regression":
            model = Logistic_Regression(300).to(device), model
        
        if model == "MLP" and dataset == "a9a":
            model = DNN( input_dim = 123, output_dim = 2).to(device), model
        if model == "MLP" and dataset == "human_activity":
            model = DNN( input_dim = 561, mid_dim = 561, output_dim = 6).to(device), model
        if model == "MLP" and dataset == "w8a":
            model = DNN( input_dim = 300, output_dim = 2).to(device), model
        if model == "MLP" and dataset == "Mnist":
            model = DNN().to(device), model
        if model == 'CNN':
            model = Net().to(device), model
        # select algorithm
        if(commet):
            experiment.set_name(dataset + "_" + algorithm + "_" + model[1] + "_" + str(batch_size) + "b_" + str(learning_rate) + "lr_" + str(alpha) + "al_" + str(eta) + "eta_" + str(L) + "L_" + str(rho) + "p_" +  str(num_glob_iters) + "ge_"+ str(local_epochs) + "le_"+ str(numedges) +"u")
        server = Server(experiment, device, dataset, algorithm, model, batch_size, learning_rate, alpha, eta,  L, num_glob_iters, local_epochs, optimizer, numedges, i)
        
        server.train()
        server.test()


In [2]:
sophia_params = {
    "dataset": "Mnist",
    "algorithm": "Sophia",
    "model": "CNN",
    "batch_size": 64,
    "learning_rate": 0.0001,
    "alpha": (0.9, 0.95),
    "eta": 1.0,
    "L": 0.001,
    "rho": 20,
    "num_glob_iters": 41,
    "local_epochs": 1,
    "optimizer": "Sophia",
    "numedges": 32,
    "times": 1,
    "commet": True,
    "gpu": 0
}



DONE_params = {
    "dataset": "human_activity",
    "algorithm": "DONE",
    "model": "MLP",
    "batch_size": 64,
    "learning_rate": 0.001,
    "alpha": 0.03,
    "eta": 1.0,
    "L": 1e-5,
    "rho": 0.001,
    "num_glob_iters": 41,
    "local_epochs": 10,
    "optimizer": "DONE",
    "numedges": 30,
    "times": 1,
    "commet": True,
    "gpu": 0
}

GD_params = {
    "dataset": "human_activity",
    "algorithm": "GD",
    "model": "MLP",
    "batch_size": 64,
    "learning_rate": 0.01,
    "alpha": 0.03,
    "eta": 20.0,
    "L": 1e-3,
    "rho": 0.01,
    "num_glob_iters": 41,
    "local_epochs": 1,
    "optimizer": "DONE",
    "numedges": 30,
    "times": 1,
    "commet": True,
    "gpu": 0
}

'''
Newton_params = {
    "dataset": "Mnist",
    "algorithm": "Newton",
    "model": "MLP",
    "batch_size": 0,
    "learning_rate": 1,
    "alpha": 0.03,
    "eta": 1.0,
    "L": 1e-3,
    "rho": 0.01,
    "num_glob_iters": 500,
    "local_epochs": 40,
    "optimizer": "Newton",
    "numedges": 30,
    "times": 1,
    "commet": True,
    "gpu": 0
}

'''

'\nNewton_params = {\n    "dataset": "Mnist",\n    "algorithm": "Newton",\n    "model": "MLP",\n    "batch_size": 0,\n    "learning_rate": 1,\n    "alpha": 0.03,\n    "eta": 1.0,\n    "L": 1e-3,\n    "rho": 0.01,\n    "num_glob_iters": 500,\n    "local_epochs": 40,\n    "optimizer": "Newton",\n    "numedges": 30,\n    "times": 1,\n    "commet": True,\n    "gpu": 0\n}\n\n'

In [3]:
experiment = Experiment(
        api_key="q24VqIVkFNEOugLA3T0YFFFvE",
        project_name="sophia",
        workspace="abdulmomen96",
    )
main(experiment, **sophia_params)
experiment.end()    


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/abdulmomen96/sophia/43690596b5654f63ad7ac4c12b1c5b5f



---------------Running time:------------ 0
Number of edges / total edges: 10  /  32


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 130480000 bytes.

In [ ]:
'''
Sophia
-------------Round number:  40  -------------
Average Global Accuracy          :  0.6722136222910217
Average Global Trainning Accuracy:  0.6764744005184705
Average Global Trainning Loss    :  1.0025231767457874


GD
-------------Round number:  40  -------------
Average Global Accuracy          :  0.20936532507739938
Average Global Trainning Accuracy:  0.20764744005184704
Average Global Trainning Loss    :  1.7220779072828905


'''

In [ ]:
from utils.plot_utils import get_training_data_value, simple_read_data

In [ ]:


rs_train_acc_sop, rs_train_loss_sop, rs_glob_acc_sop = simple_read_data("human_activity_Sophia_0.0001_(0.9, 0.95)_1.0_0.001_30u_64b_1_0")
rs_train_acc_gd, rs_train_loss_gd, rs_glob_acc_gd = simple_read_data("human_activity_GD_0.01_0.03_20.0_0.001_30u_64b_1_0")

In [ ]:

import matplotlib.pyplot as plt


# Create a figure with two subplots
plt.figure(figsize=(10, 5))

# Subplot 1: Training Accuracy
plt.subplot(1, 2, 1)
plt.plot(rs_train_acc_sop, label='FedSophaia')
plt.plot(rs_train_acc_gd, label='GD')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Subplot 2: Training Loss
plt.subplot(1, 2, 2)
plt.plot(rs_train_loss_sop, label='FedSophia')
plt.plot(rs_train_loss_gd, label='GD')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


plt.tight_layout()
plt.show()
